In [ ]:
import pandas as pd
import QuantLib as ql
from QuantLib import YieldTermStructureHandle
import numpy as np
from scipy.optimize import minimize


In [2]:
kalendarz_odsetkowy = pd.read_parquet(
    "data/kalendarz_odsetkowy.parquet",
    columns=["Seria", "Kod ISIN", "Koniec okresu", "Początek okresu", "Kupon", "Data wykupu"],
)
ceny_rentownosci = pd.read_parquet(
    "data/bond_prices.parquet",
    columns=["Seria", "Kod ISIN", "Fixing", "fix_price", "Date"],
)

data = ceny_rentownosci.merge(kalendarz_odsetkowy, on=["Seria", "Kod ISIN"])
data = data.loc[(data["Koniec okresu"] >= data.Date) & (data.Date > data["Początek okresu"])]
data = (
    data.set_index(["Seria", "Date", "Fixing"])
    .sort_index(ascending=True)
    .ffill()
    .reset_index(level="Fixing")
    .ffill()
    .reset_index()
    .drop_duplicates(subset=["Seria", "Date"], keep="last")
    .set_index(["Date", "Seria"])
    .sort_index()
)

dates = data.index.unique(0)

In [3]:
def build_zero_curve_from_bonds(
    bonds: pd.DataFrame,
    eval_date,
    calendar=ql.Poland(),
    day_counter=ql.ActualActual(ql.ActualActual.ISMA),
    settlement_days=2,
    curve_cls=ql.PiecewiseCubicZero,
) -> YieldTermStructureHandle:
    eval_date_ql = ql.Date(eval_date.day, eval_date.month, eval_date.year)

    ql.Settings.instance().evaluationDate = eval_date_ql

    helpers = []
    for _, (price, coupon, maturity, start) in bonds.iterrows():
        start = ql.Date(start.day, start.month, start.year)
        maturity = ql.Date(maturity.day, maturity.month, maturity.year)

        schedule = ql.MakeSchedule(
            start,
            maturity,
            ql.Period(ql.Annual),
            calendar=calendar,
            convention=ql.ModifiedFollowing,
            terminalDateConvention=ql.ModifiedFollowing,
            rule=ql.DateGeneration.Backward,
            endOfMonth=False,
        )

        quote = ql.QuoteHandle(ql.SimpleQuote(price))
        helper = ql.FixedRateBondHelper(
            quote,
            settlement_days,
            100.0,
            schedule,
            [coupon],
            day_counter,
            ql.Following,
            100.0,
            start,
        )
        helpers.append(helper)

    curve = curve_cls(eval_date_ql, helpers, day_counter)
    return ql.YieldTermStructureHandle(curve)

In [4]:
def get_zero_rates_from_curve(
    curve,
    eval_date,
    periods=(0, 1, 2, 3, 6, 12, 24, 36, 60, 84, 120, 180),
):
    eval_date_ql = ql.Date(eval_date.day, eval_date.month, eval_date.year)
    rates = []
    for period in periods:
        moved_date = eval_date_ql + ql.Period(period, ql.Months)
        try:
            rate = curve.zeroRate(moved_date, ql.ActualActual(ql.ActualActual.ISMA), ql.Continuous)
        except Exception:
            continue
        else:
            rates.append((rate.rate(), eval_date, moved_date.to_date(), period))

    return rates

In [5]:
rates = []

for eval_date in dates:
    bonds = data.loc[eval_date, ["fix_price", "Kupon", "Data wykupu", "Początek okresu"]]
    curve = build_zero_curve_from_bonds(bonds, eval_date)

    rate = get_zero_rates_from_curve(curve, eval_date)
    rates.extend(rate)


In [6]:
zero_rates = (
    pd.DataFrame(rates, columns=["rate", "eval_date", "future_date", "period"])
    .drop_duplicates()
    .astype({"future_date": "datetime64[ns]"})
    .set_index(["eval_date", "future_date"])
    .sort_index()
)
zero_rates

rate  period
eval_date  future_date                  
2011-01-03 2011-01-03   0.041606       0
           2011-02-03   0.041606       1
           2011-03-03   0.041606       2
           2011-04-03   0.041606       3
           2011-07-03   0.041606       6
...                          ...     ...
2025-09-19 2027-09-19   0.042364      24
           2028-09-19   0.044631      36
           2030-09-19   0.048257      60
           2032-09-19   0.050817      84
           2035-09-19   0.054747     120

[40271 rows x 2 columns]

In [7]:
def nss(params, periods):
    beta0, beta1, beta2, beta3, tau1, tau2 = params

    return (
        (beta0)
        + (beta1 * (-np.expm1(-periods / tau1) / (periods / tau1)))
        + (beta2 * (-(np.expm1(-periods / tau1) / (periods / tau1)) - (np.exp(-periods / tau1))))
        + (beta3 * (-(np.expm1(-periods / tau2) / (periods / tau2)) - (np.exp(-periods / tau2))))
    )


In [8]:
def objective_function(initial_guess, df):
    nss_ = nss(initial_guess, df.period)
    return ((df.rate - nss_) ** 2).sum()

In [9]:
vals = []
for idx in dates:
    df = zero_rates.loc[idx]
    c = minimize(
        objective_function,
        [0.01, 0.01, 0.01, 0.01, 1.00, 1.00],
        args=(df,),
    )
    vals.append(c)

In [10]:
params = pd.DataFrame(
    [val.x for val in vals],
    columns=["beta0", "beta1", "beta2", "beta3", "tau1", "tau2"],
    index=dates,
)
params.describe()

,beta0,beta1,beta2,beta3,tau1,tau2
count,3669.000000,3669.000000,3669.000000,3669.000000,3669.000000,3669.000000
mean,0.043612,-0.014696,-0.035860,0.011382,12.506154,12.501335
std,0.020850,0.019593,0.105884,0.106335,20.057728,20.061666
min,0.003345,-0.537089,-3.092057,-2.407573,0.999783,0.999911
25%,0.035196,-0.025947,-0.093528,-0.022160,1.000055,1.000116
50%,0.042638,-0.018477,-0.012863,-0.009095,13.211601,13.202474
75%,0.053687,0.001671,0.001703,0.067566,19.191892,19.191490
max,0.544008,0.022045,2.019100,2.779652,735.659048,735.126431


In [ ]:
months = np.arange(1, 181)

curves = params.apply(nss, args=(months,), axis="columns", result_type="expand")
curves.columns = months
full = params.join(curves)
full.to_parquet("data/nss_curve.parquet")
full

c:\Users\alans\REPOS\MyQuantLibNotebooks\.venv\Lib\site-packages\pandas\io\parquet.py:191: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


,beta0,beta1,beta2,beta3,tau1,tau2,1,2,3,4,...,171,172,173,174,175,176,177,178,179,180
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,0.065990,-0.023903,0.012001,-0.044511,14.002670,14.027300,0.041817,0.041609,0.041458,0.041359,...,0.061365,0.061392,0.061418,0.061444,0.061470,0.061496,0.061521,0.061547,0.061571,0.061596
2011-01-04,0.066065,-0.023874,0.012696,-0.045014,14.115612,14.140483,0.041927,0.041725,0.041579,0.041484,...,0.061420,0.061447,0.061474,0.061500,0.061526,0.061552,0.061577,0.061603,0.061628,0.061652
2011-01-05,0.065730,-0.022596,0.014856,-0.044824,13.540680,13.564523,0.042897,0.042721,0.042600,0.042529,...,0.061561,0.061585,0.061609,0.061633,0.061656,0.061679,0.061702,0.061725,0.061747,0.061769
2011-01-07,0.065908,-0.023331,0.012903,-0.042990,13.643448,13.667458,0.042364,0.042210,0.042110,0.042058,...,0.061641,0.061665,0.061690,0.061714,0.061738,0.061762,0.061785,0.061808,0.061831,0.061854
2011-01-10,0.066153,-0.021054,0.035080,-0.058320,16.135527,16.163713,0.045050,0.045030,0.045037,0.045069,...,0.061965,0.061989,0.062013,0.062037,0.062060,0.062083,0.062106,0.062129,0.062151,0.062174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-15,0.061079,-0.024838,0.019317,-0.038545,19.472825,19.496061,0.036392,0.036553,0.036725,0.036905,...,0.056059,0.056088,0.056117,0.056145,0.056173,0.056201,0.056228,0.056255,0.056282,0.056309
2025-09-16,0.065158,-0.030429,0.049163,-0.050578,44.163907,44.216233,0.035055,0.035377,0.035695,0.036008,...,0.057125,0.057167,0.057209,0.057251,0.057292,0.057333,0.057373,0.057413,0.057453,0.057492
2025-09-17,0.065411,-0.030391,0.056980,-0.057198,47.158700,47.215119,0.035339,0.035653,0.035962,0.036267,...,0.057190,0.057233,0.057276,0.057318,0.057359,0.057401,0.057442,0.057482,0.057523,0.057563
